In [84]:
class PETs_eval_anonymeter():
    import PETs_Tool

    def __init__(self
                ,data_ori ,data_syn ,data_control
                ,params={} ,**kwargs):
        ####### ####### #######
        # init - params       #
        ####### ####### #######
        # {'evaluate_params' : {'anonymeter':
        default_params = {'SinglingOut' : {'univariate'   : 'Y'
                                          ,'multivariate' : 'N'
                                          }
                         ,'SinglingOut_params' : {'univariate'   : {'n_attacks' : 500}
                                                 ,'multivariate' : {'n_attacks' : 500
                                                                   ,'n_cols'    : 2
                                                                   }
                                                 }
                         ,'Linkability' : 'Y'
                         ,'Linkability_params' : {'n_attacks'   : 2000
                                                 ,'aux_cols'    : []
                                                 # [TODO] study how to set n_neighbors
                                                 ,'n_neighbors' : 10
                                                 # n_jobs follow joblib convention.
                                                 # -1 = all cores,
                                                 # -2 = all execept one
                                                 ,'n_jobs'      : -2
                                                 }
                         ,'Inference' : 'Y'
                         ,'Inference_params' : {'n_attacks'   : 1000
                                               ,'secret'      : ''
                                               # n_jobs follow joblib convention.
                                               # -1 = all cores,
                                               # -2 = all execept one
                                               ,'n_jobs'      : -2
                                               }
                         }
        self.params = PETs_Tool.PETs_util.update_append_nested(default_params ,params)
        self.data_ori     = data_ori
        self.data_syn     = data_syn
        self.data_control = data_control
        self.evaluators   = {}
        self.results      = {}

        import warnings
        with warnings.catch_warnings():
            # anonymeter\evaluators\singling_out_evaluator.py:97:
            # FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version.
            # Use isinstance(dtype, CategoricalDtype) instead elif is_categorical_dtype(values)
            warnings.simplefilter("ignore" ,category=FutureWarning)
            # c:\Program Files\Python310\lib\site-packages\anonymeter\stats\confidence.py:215:
            # UserWarning: Attack is as good or worse as baseline model.
            # Estimated rates: attack = 0.30674239114619767, baseline = 0.30773856438771213.
            # Analysis results cannot be trusted. self._sanity_check()
            warnings.simplefilter("ignore" ,category=UserWarning)

            ####### ####### #######
            # init - SinglingOut  #
            ####### ####### #######
            if any(v == 'Y' for v in self.params['SinglingOut'].values()):
                self.evaluators['SinglingOut'] = {}
                self.results[   'SinglingOut'] = {}

                if self.params['SinglingOut']['univariate'] == 'Y':
                    self.__SinglingOutEvaluator(mode = 'univariate'
                                               ,**self.params['SinglingOut_params']['univariate']
                                               )
                # if self.params['SinglingOut']['multivariate'] == 'Y':
                #     self.__SinglingOutEvaluator(mode = 'multivariate'
                #                                ,**self.params['SinglingOut_params']['multivariate']
                #                                )
            ####### ####### #######
            # init - SinglingOut  #
            ####### ####### #######
            if self.params['Linkability'] == 'Y':
                self.__LinkabilityEvaluator(**self.params['Linkability_params'])


            if self.params['Inference'] == 'Y':
                self.evaluators['Inference'] = {}
                self.results[   'Inference'] = {}
                if self.params['Inference_params']['secret'] == '':
                    for __secret in self.data_syn.columns:
                        self.params['Inference_params']['secret'] = __secret
                        self.__InferenceEvaluator(**self.params['Inference_params'])
                else:
                    self.__InferenceEvaluator(**self.params['Inference_params'])

            



    ####### ####### #######
    # SinglingOut         #
    ####### ####### #######
    def __SinglingOutEvaluator(self ,mode ,**kwargs):
        import time
        __time_start = time.time()
        print('Now is Singling-Out Evaluator')
        from anonymeter import evaluators
        __evaluator = evaluators.SinglingOutEvaluator(ori       = self.data_ori
                                                     ,syn       = self.data_syn
                                                     ,control   = self.data_control
                                                     ,**kwargs
                                                     )
        try:
            __evaluator.evaluate(mode=mode)
            self.evaluators['SinglingOut'][mode] = __evaluator
            self.results[   'SinglingOut'][mode] = self.__Result(self.evaluators['SinglingOut'][mode])
        except RuntimeError as ex:
            print(f"Singling out evaluation failed with {ex}."
                   "Please re-run this cell."
                   "For more stable results increase `n_attacks`. Note that this will "
                   "make the evaluation slower.")
        # multivariance
        # try:
        #     __evaluator.evaluate(mode='multivariate')
        #     __risk = __evaluator.risk()
        #     print(__risk)
        #     res = __evaluator.results()
        #     print(__evaluator.queries()[:3])
        # except RuntimeError as ex:
        #     print(f"Singling out evaluation failed with {ex}. Please re-run this cell."
        #           "For more stable results increase `n_attacks`. Note that this will "
        #           "make the evaluation slower.")
        print(f"Singling-Out Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Linkability         #
    ####### ####### #######
    def __LinkabilityEvaluator(self ,n_jobs ,aux_cols ,**kwargs):
        import time
        __time_start = time.time()
        print('Now is Linkability Evaluator')
        from anonymeter import evaluators
        __str_aux_cols = "\nand ".join(f"[{', '.join(row)}]" for row in aux_cols)
        print(f"aux_cols are {__str_aux_cols}.")
        __evaluator = evaluators.LinkabilityEvaluator(ori      = self.data_ori
                                                     ,syn      = self.data_syn
                                                     ,control  = self.data_control
                                                     ,aux_cols = aux_cols
                                                     ,**kwargs
                                                     )
        __evaluator.evaluate(n_jobs=n_jobs)
        self.evaluators['Linkability'] = __evaluator
        self.results[   'Linkability'] = self.__Result(self.evaluators['Linkability'])
        print(f"Linkability Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Inference           #
    ####### ####### #######
    def __InferenceEvaluator(self ,secret ,n_jobs ,**kwargs):
        import time
        __time_start = time.time()
        print(f"Now is Inference Evaluator: secret is {secret}.")
        from anonymeter import evaluators
        __aux_cols = [col for col in self.data_syn.columns if col != secret]
        __evaluator = evaluators.InferenceEvaluator(ori      = self.data_ori
                                                   ,syn      = self.data_syn
                                                   ,control  = self.data_control
                                                   ,aux_cols = __aux_cols
                                                   ,secret   = secret
                                                   ,**kwargs
                                                   )
        __evaluator.evaluate(n_jobs=n_jobs)
        self.evaluators['Inference'][secret] = __evaluator
        self.results[   'Inference'][secret] = self.__Result(self.evaluators['Inference'][secret])
        print(f"Inference Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Result              #
    ####### ####### #######
    def __Result(self ,__evaluator): 
        import numpy as np
        __dict_result    = {}
        __para_to_handle = [('Risk'              , ['risk()'    ,'value'                 ])
                           ,('Risk_CI_btm'       , ['risk()'    ,'ci[0]'                 ])
                           ,('Risk_CI_top'       , ['risk()'    ,'ci[1]'                 ])
                           ,('Attack_Rate'       , ['results()' ,'attack_rate'   ,'value'])
                           ,('Attack_Rate_err'   , ['results()' ,'attack_rate'   ,'error'])
                           ,('Baseline_Rate'     , ['results()' ,'baseline_rate' ,'value'])
                           ,('Baseline_Rate_err' , ['results()' ,'baseline_rate' ,'error'])
                           ,('Control_Rate'      , ['results()' ,'control_rate'  ,'value'])
                           ,('Control_Rate_err'  , ['results()' ,'control_rate'  ,'error'])
                           ]
        for __key ,__attrs in __para_to_handle:
            try:
                __attr_value = __evaluator
                for __attr in __attrs:
                    if '()' in __attr:
                        __method_name = __attr.split('(')[0]
                        if hasattr(__attr_value ,__method_name):
                            __method = getattr(__attr_value ,__method_name)
                            if callable(__method):
                                __attr_value = __method()
                            else:
                                __dict_result[__key] = np.nan
                                break
                        else:
                            __dict_result[__key] = np.nan
                            break
                    elif '[' in __attr:
                        __attr_name = __attr.split('[')[0]
                        __index = int(__attr.split('[')[1].rstrip(']'))
                        if hasattr(__attr_value, __attr_name)\
                        and isinstance(getattr(__attr_value, __attr_name) ,(list, dict, tuple)):
                            try:
                                __attr_value = getattr(__attr_value  ,__attr_name)[__index]
                            except (IndexError, KeyError):
                                __dict_result[__key] = np.nan
                                break
                        else:
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __attr_value = getattr(__attr_value, __attr)
                __dict_result[__key] = __attr_value
            except Exception as e:
                __dict_result[__key] = np.nan
        return __dict_result

In [91]:
import os
import PETs_Tool


folder_raw = '.\\data'
folder_SD  = '.\\data_dpsd'



dict_PETs_setting = {'adult' : {'filename'    : '[Adult]'
                               ,'read_params' : {'downcast'       : 'N' # 'Y'
                                                ,'label_encoding' : 'Y'}
                            #    ,'describe_params' : {'data': {'?'}}
                               }
                    }
dict_filename = {'adult': {'raw' : '[Adt Income] adult.csv'
                          }
                }
dict_filename = {key: {**value, 'params': dict_PETs_setting.get(key, {})} 
                 for key, value in dict_filename.items()}
# print(dict_filename)



# update default in SDV
list_adult_SD_SDV_train = ['GaussianCoupula'
                        #    'CoupulaGAN'
                        #   ,'CTGAN'
                        #   ,'GaussianCoupula'
                        #   ,'TVAE'
                          ]
dict_adult_SD_SDV_train = {k: {'sd_params': {'model': k}} for k in list_adult_SD_SDV_train}
dict_filename = {key: PETs_Tool.PETs_util.update_append_nested(value
                                                              ,{'SD_train': dict_adult_SD_SDV_train}
                                                              ) 
                 for key, value in dict_filename.items()}
dict_adult_SD_SDV_train_default = {'sd_params': {'save_model': 'N'}
                                  ,'sample'   : 'Y'
                                  ,'sample_params' : {'sample_rows_as_raw' : 'Y'
                                                     ,'save_data' : 'N'}
                                  }
dict_adult_SD_train = {'SDV' : {key: PETs_Tool.PETs_util.update_append_nested(dict_adult_SD_SDV_train_default
                                                                       ,dict_adult_SD_SDV_train.get(key ,{})
                                                                       )
                                for key in dict_adult_SD_SDV_train
                               }
                      }
dict_filename = {k1: {k2: dict_adult_SD_train if k2 == 'SD_train' else v2
                      for k2 ,v2 in dict_filename[k1].items()
                     } 
                 for k1 in dict_filename
                }
# print(dict_filename)


same_sample_times = 5
dict_bootstrap = {'bootstrap_params' : {'bootstrap_time' : 5 # 3 # 30
                                       ,'sample_ratio'   : 0.8
                                       ,'random_state'   : None
                                       }
                 ,'index_params' : {'index_save'      : 'N'
                                   ,'index_save_data' : 'N'
                                   ,'index_filename'  : 'Anomymeter'
                                   }
                 }
# import json
# print(json.dumps(dict_bootstrap ,indent=4))

__Synther_param = {'sd_params': {'metadata' : 'SingleTable'
                                ,'save_model' : 'N'}
                  ,'sample' : 'Y'
                  ,'sample_params' : {'sample_rows_as_raw' : 'Y'
                                     ,'save_data' : 'N'}
                  }
aux_cols = [['age' ,'fnlwgt' ,'race' ,'gender' ,'native-country']
           ,['workclass' ,'education' ,'capital-gain' ,'capital-loss' ,'hours-per-week']
           ]

__param_anonymeter = {'SinglingOut': {'univariate'   : 'Y'
                                     ,'multivariate' : 'N'
                                     }
                     ,'Linkability' : 'Y'
                     ,'Linkability_params' : {'aux_cols' : aux_cols}
                     ,'Inference' : 'Y'
                     ,'Inference_params' : {'secret' : ''} # all columns
                     }
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for key, value in d.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.extend(flatten_dict(value, new_key, sep=sep).items())
        else:
            items.append((new_key, value))
    return dict(items)






__dict_anonymeter_result = {}
for __dataset ,v in dict_filename.items():
    __filename = v['params']['filename']
    print(f"Now is {__filename}: {v['raw']} comparision.")

    __Loader = PETs_Tool.PETs_Loader(os.path.join(folder_raw ,v['raw'])
                                    ,v['params']
                                    )
    __df_ori = __Loader.data
    print(f"Now is {__filename} original data been load: {v['raw']}.")

    dict_boostrap_index = PETs_Tool.PETs_util.df_bootstrap(__df_ori ,dict_bootstrap)
    digits_max_boostrap = len(str(max(dict_boostrap_index.keys()))) # max boostrap times for fill zero
    digits_same_sample  = len(str(    same_sample_times          )) # same sample times for fill zero         

    for __library ,__dict_sd in v['SD_train'].items():
        print(f"Now is {__filename} by library {__library}.")

        for __method ,__params in __dict_sd.items():
            print(f"Now is {__filename} by method {__method} in library {__library}.")

            import copy
            for time ,dict_idx in dict_boostrap_index.items():
                __trail_name = f"{__filename}_{__library}_{__method}_{str(time+1).zfill(digits_max_boostrap)}"
                print(f"Trail as {__trail_name}.")

                ####### ####### #######
                # init - ori ,ctrl ,syn #
                ####### ####### #######
                __Synther = copy.deepcopy(__Loader)
                __data = __Synther.data.copy()
                __Synther.data_ori     = __data.loc[dict_boostrap_index[time]['idx_train'     ]].reset_index(drop=True)
                __Synther.data_control = __data.loc[dict_boostrap_index[time]['idx_validation']].reset_index(drop=True)
                __Synther.data         = __Synther.data_ori.copy()
                __Synther.data_syn     = PETs_Tool.PETs_SD_SDV(__Synther ,__Synther_param).synthetic_data
                __Synther.data = __data.copy()

                for same_sample_time in range(same_sample_times):
                    __trail_redo_name = f"{__trail_name}_{str(same_sample_time+1).zfill(digits_same_sample)}"
                    print(f"Trail x Redo as {__trail_redo_name}.")
                    ####### ####### #######
                    # _anonymeter         #
                    ####### ####### #######
                    __anonymeter = PETs_eval_anonymeter(data_ori     = __Synther.data_ori
                                                       ,data_syn     = __Synther.data_syn
                                                       ,data_control = __Synther.data_control
                                                       ,params = __param_anonymeter
                                                       )
                    __dict_anonymeter_result[__trail_redo_name] = {}
                    for __result_key ,__result_value in __anonymeter.results.items():
                        __dict_anonymeter_result[__trail_redo_name].update(flatten_dict(__result_value ,parent_key=__result_key))

                # 20231116, Justyn: refer to
                # https://github.com/statice/anonymeter/blob/main/notebooks/anonymeter_example.ipynb
                # https://storage.googleapis.com/statice-public/anonymeter-datasets

    import pandas as pd

    __df_result = pd.DataFrame.from_dict(__dict_anonymeter_result ,orient='index')
    # [TODO] 按照 SinglingOut/Linkability/Inference 以及 colname 排序

    from datetime import datetime
    import pytz
    __desctime = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    __df_result.to_csv('Justyn_anonymeter_20231129_1130.csv')



Now is [Adult]: [Adt Income] adult.csv comparision.
Now is [Adult] original data been load: [Adt Income] adult.csv.
Bootstrap df: 5 times is done.
Now is [Adult] by library SDV.
Now is [Adult] by method GaussianCoupula in library SDV.
Trail as [Adult]_SDV_GaussianCoupula_1.
Metafile loading time: 0.1287 sec.
We are execute SingleTable - GaussianCoupula.
Model training time: 19.7345 sec.
Sample as same as raw data: # 48842 rows data by GaussianCoupula in 2.2266 秒
Trail as [Adult]_SDV_GaussianCoupula_1_1.
Now is Singling-Out Evaluator


Found 195 failed queries out of 500. Check DEBUG messages for more details.


Singling-Out Evaluator time: 32.2437 sec.
Now is Linkability Evaluator
aux_cols are [age, fnlwgt, race, gender, native-country]
and [workclass, education, capital-gain, capital-loss, hours-per-week].
Linkability Evaluator time: 7.6582 sec.
Now is Inference Evaluator: secret is age.
Inference Evaluator time: 2.8412 sec.
Now is Inference Evaluator: secret is workclass.
Inference Evaluator time: 3.161 sec.
Now is Inference Evaluator: secret is fnlwgt.
Inference Evaluator time: 2.4642 sec.
Now is Inference Evaluator: secret is education.
Inference Evaluator time: 2.3072 sec.
Now is Inference Evaluator: secret is educational-num.
Inference Evaluator time: 2.2594 sec.
Now is Inference Evaluator: secret is marital-status.
Inference Evaluator time: 2.1964 sec.
Now is Inference Evaluator: secret is occupation.
Inference Evaluator time: 2.2594 sec.
Now is Inference Evaluator: secret is relationship.
Inference Evaluator time: 2.3122 sec.
Now is Inference Evaluator: secret is race.
Inference Evalu

Found 197 failed queries out of 500. Check DEBUG messages for more details.


Singling-Out Evaluator time: 33.2577 sec.
Now is Linkability Evaluator
aux_cols are [age, fnlwgt, race, gender, native-country]
and [workclass, education, capital-gain, capital-loss, hours-per-week].
Linkability Evaluator time: 7.7605 sec.
Now is Inference Evaluator: secret is age.
Inference Evaluator time: 2.354 sec.
Now is Inference Evaluator: secret is workclass.
Inference Evaluator time: 2.4171 sec.
Now is Inference Evaluator: secret is fnlwgt.
Inference Evaluator time: 2.4172 sec.
Now is Inference Evaluator: secret is education.
Inference Evaluator time: 2.37 sec.
Now is Inference Evaluator: secret is educational-num.
Inference Evaluator time: 2.4953 sec.
Now is Inference Evaluator: secret is marital-status.
Inference Evaluator time: 2.4996 sec.
Now is Inference Evaluator: secret is occupation.
Inference Evaluator time: 2.6061 sec.
Now is Inference Evaluator: secret is relationship.
Inference Evaluator time: 2.5062 sec.
Now is Inference Evaluator: secret is race.
Inference Evaluat

...

Singling-Out Evaluator time: 32.9942 sec.
Now is Linkability Evaluator
aux_cols are [age, fnlwgt, race, gender, native-country]
and [workclass, education, capital-gain, capital-loss, hours-per-week].
Linkability Evaluator time: 7.2867 sec.
Now is Inference Evaluator: secret is age.
Inference Evaluator time: 2.418 sec.
Now is Inference Evaluator: secret is workclass.
Inference Evaluator time: 2.3678 sec.
Now is Inference Evaluator: secret is fnlwgt.
Inference Evaluator time: 2.4971 sec.
Now is Inference Evaluator: secret is education.
Inference Evaluator time: 2.4607 sec.
Now is Inference Evaluator: secret is educational-num.
Inference Evaluator time: 2.3345 sec.
Now is Inference Evaluator: secret is marital-status.
Inference Evaluator time: 2.4454 sec.
Now is Inference Evaluator: secret is occupation.
Inference Evaluator time: 2.5245 sec.
Now is Inference Evaluator: secret is relationship.
Inference Evaluator time: 2.612 sec.
Now is Inference Evaluator: secret is race.
Inference Evalua

Found 198 failed queries out of 500. Check DEBUG messages for more details.


Singling-Out Evaluator time: 32.2621 sec.
Now is Linkability Evaluator
aux_cols are [age, fnlwgt, race, gender, native-country]
and [workclass, education, capital-gain, capital-loss, hours-per-week].
Linkability Evaluator time: 7.0697 sec.
Now is Inference Evaluator: secret is age.
Inference Evaluator time: 2.4345 sec.
Now is Inference Evaluator: secret is workclass.
Inference Evaluator time: 3.2362 sec.
Now is Inference Evaluator: secret is fnlwgt.
Inference Evaluator time: 3.5457 sec.
Now is Inference Evaluator: secret is education.
Inference Evaluator time: 3.1728 sec.
Now is Inference Evaluator: secret is educational-num.
Inference Evaluator time: 3.2884 sec.
Now is Inference Evaluator: secret is marital-status.
Inference Evaluator time: 4.3483 sec.
Now is Inference Evaluator: secret is occupation.
Inference Evaluator time: 3.2895 sec.
Now is Inference Evaluator: secret is relationship.
Inference Evaluator time: 2.9302 sec.
Now is Inference Evaluator: secret is race.
Inference Eval

Found 187 failed queries out of 500. Check DEBUG messages for more details.


Singling-Out Evaluator time: 32.8427 sec.
Now is Linkability Evaluator
aux_cols are [age, fnlwgt, race, gender, native-country]
and [workclass, education, capital-gain, capital-loss, hours-per-week].
Linkability Evaluator time: 7.3707 sec.
Now is Inference Evaluator: secret is age.
Inference Evaluator time: 2.3501 sec.
Now is Inference Evaluator: secret is workclass.
Inference Evaluator time: 2.4515 sec.
Now is Inference Evaluator: secret is fnlwgt.
Inference Evaluator time: 2.5476 sec.
Now is Inference Evaluator: secret is education.
Inference Evaluator time: 2.4754 sec.
Now is Inference Evaluator: secret is educational-num.
Inference Evaluator time: 2.4653 sec.
Now is Inference Evaluator: secret is marital-status.
Inference Evaluator time: 2.4164 sec.
Now is Inference Evaluator: secret is occupation.
Inference Evaluator time: 2.5257 sec.
Now is Inference Evaluator: secret is relationship.
Inference Evaluator time: 2.4679 sec.
Now is Inference Evaluator: secret is race.
Inference Eval

In [93]:
import pandas as pd
pd.DataFrame.from_dict(__dict_anonymeter_result ,orient='index')

,SinglingOut_univariate_Risk,SinglingOut_univariate_Risk_CI_btm,SinglingOut_univariate_Risk_CI_top,SinglingOut_univariate_Attack_Rate,SinglingOut_univariate_Attack_Rate_err,SinglingOut_univariate_Baseline_Rate,SinglingOut_univariate_Baseline_Rate_err,SinglingOut_univariate_Control_Rate,SinglingOut_univariate_Control_Rate_err,Linkability_Risk,...,Inference_native-country_Control_Rate_err,Inference_income_Risk,Inference_income_Risk_CI_btm,Inference_income_Risk_CI_top,Inference_income_Attack_Rate,Inference_income_Attack_Rate_err,Inference_income_Baseline_Rate,Inference_income_Baseline_Rate_err,Inference_income_Control_Rate,Inference_income_Control_Rate_err
[Adult]_SDV_GaussianCoupula_1_1,0.039959,0.019697,0.060221,0.049461,0.018616,0.003812,0.003812,0.009898,0.007788,0.001000,...,0.011034,0.062127,0.0,0.184381,0.699235,0.028359,0.662376,0.029247,0.679311,0.028865
[Adult]_SDV_GaussianCoupula_1_1_2,0.035712,0.012503,0.058922,0.055416,0.019685,0.005797,0.005444,0.020433,0.011796,0.000500,...,0.013336,0.105122,0.0,0.225992,0.720154,0.027758,0.612568,0.030133,0.687281,0.028670
[Adult]_SDV_GaussianCoupula_1_1_2_3,0.019705,0.000000,0.043887,0.049461,0.018616,0.003812,0.003812,0.030354,0.014543,0.001998,...,0.011959,0.072387,0.0,0.198887,0.719158,0.027788,0.623525,0.029968,0.697242,0.028412
[Adult]_SDV_GaussianCoupula_1_1_2_3_4,0.024624,0.002754,0.046495,0.045492,0.017860,0.003812,0.003812,0.021394,0.012092,0.000000,...,0.009641,0.000000,0.0,0.140623,0.721150,0.027727,0.638468,0.029716,0.721150,0.027727
[Adult]_SDV_GaussianCoupula_1_1_2_3_4_5,0.038444,0.017874,0.059014,0.049461,0.018616,0.003812,0.003812,0.011458,0.008507,0.003000,...,0.012249,0.124172,0.0,0.250191,0.747051,0.026874,0.621533,0.029999,0.711189,0.028024
[Adult]_SDV_GaussianCoupula_2_1,0.000000,0.000000,0.027506,0.051446,0.018981,0.005797,0.005444,0.052353,0.019145,0.003998,...,0.013206,0.000000,0.0,0.092794,0.680307,0.028841,0.647434,0.029550,0.693258,0.028517
[Adult]_SDV_GaussianCoupula_2_1_2_3,0.049013,0.026950,0.071077,0.059385,0.020360,0.003812,0.003812,0.010906,0.008260,0.000000,...,0.013717,0.035153,0.0,0.162184,0.699235,0.028359,0.643449,0.029625,0.688277,0.028645
[Adult]_SDV_GaussianCoupula_2_1_2_3_4,0.000000,0.000000,0.016187,0.055416,0.019685,0.003812,0.003812,0.070197,0.022064,0.000501,...,0.012807,0.000000,0.0,0.085892,0.689273,0.028619,0.637472,0.029734,0.705212,0.028194
[Adult]_SDV_GaussianCoupula_2_1_2_3_4_5,0.036822,0.018238,0.055407,0.043507,0.017466,0.005797,0.005444,0.006940,0.006189,0.000000,...,0.012105,0.089194,0.0,0.211410,0.715173,0.027908,0.632491,0.029820,0.687281,0.028670
[Adult]_SDV_GaussianCoupula_3_1,0.009127,0.000000,0.030443,0.035568,0.015777,0.003812,0.003812,0.026685,0.013598,0.001000,...,0.011507,0.064119,0.0,0.189010,0.709196,0.028082,0.641457,0.029662,0.689273,0.028619


- 這是已經有檔案的做法 (Unfinished)

In [ ]:
dict_adult_SD_SDV = {'CoupulaGAN'      : ['[Adt Income]_[SDV]_[CoupulaGAN]_20231106_113038_Asia_Taipei_10000.csv']
                    ,'CTGAN'           : ['[Adt Income]_[SDV]_[CTGAN]_20231106_093702_Asia_Taipei_10000.csv']
                    ,'GaussianCoupula' : ['[Adt Income]_[SDV]_[GaussianCoupula]_20231106_093622_Asia_Taipei_10000.csv']
                    ,'TVAE'            : ['[Adt Income]_[SDV]_[TVAE]_20231106_105252_Asia_Taipei_10000.csv']
                    }
dict_filename = {key: {**value ,'SDV': dict_adult_SD_SDV} 
                 for key, value in dict_filename.items()}



for __dataset ,v in dict_filename.items():
    __filename = v['params']['filename']
    print(f"Now is {__filename}: {v['raw']} comparision.")

    __df_ori = PETs_Tool.PETs_Loader(os.path.join(folder_raw ,v['raw'])
                                    ,v['params']
                                    ).data
    print(f"Now is {__filename} original data been load: {v['raw']}.")

    for __library ,__dict_sd in v['SD'].items():
        print(f"Now is {__filename} by library {__library}.")

        for __method ,__list_sd in __dict_sd.items():
            print(f"Now is {__filename} by method {__method} in library {__library}.")
            __list_sd = [__list_sd] if isinstance(__list_sd ,str) else __list_sd
            for __filename_sd in __list_sd:
                # 20231116, Justyn: 這裡是合成資料有落地
                #                   如果必要，可以直接調用 PETs_SD_* 結果的.synthetic_data
                __df_syn = PETs_Tool.PETs_Loader(os.path.join(folder_SD ,__filename_sd)
                                                ,v['params']
                                                ).data
                print(f"Now is {__filename} synthetic data been load: {__filename_sd}.")

                # 20231116, Justyn: refer to
                # https://github.com/statice/anonymeter/blob/main/notebooks/anonymeter_example.ipynb
                # https://storage.googleapis.com/statice-public/anonymeter-datasets

